## Example 1: defprod (Product Types)

Define a product type and prove properties:

```lisp
;; Define a sandwich with bread (integer) and fillings (symbol)
(defprod sandwich
  ((bread integerp)
   (fillings symbolp)))

;; Helper function
(defun x^2+y^2 (x y) (+ (* x x) (* y y)))

;; Prove that square sum of bread fields is non-negative
(defthm fty-defprod-theorem
  (implies (and (sandwich-p s1)
                (sandwich-p s2))
           (>= (x^2+y^2
                (sandwich->bread s1)
                (sandwich->bread s2))
               0))
  :hints(("Goal"
          :smtlink
          (:fty (sandwich))))
  :rule-classes nil)
```

### Key Point: `:fty` Hint

The `:fty (sandwich)` hint tells SMTLink to look up the `sandwich` type in `fty::flextypes-table` and translate it appropriately for Z3.

## Counter-Examples for defprod

When a theorem fails, SMTLink provides structured counter-examples:

```
Possible counter-example found:
((S2 (SANDWICH 0 (SYM 2))) (S1 (SANDWICH 0 (SYM 1))))
```

This shows concrete `sandwich` values that falsify the conjecture.

## Example 2: deflist (Typed Lists)

```lisp
;; Use ACL2's built-in integer-list
(defthm fty-deflist-theorem
  (implies (and (integer-listp l)
                (consp (acl2::integer-list-fix l))
                (consp (acl2::integer-list-fix 
                        (cdr (acl2::integer-list-fix l)))))
           (>= (x^2+y^2 
                (car (acl2::integer-list-fix l))
                (car (acl2::integer-list-fix
                      (cdr (acl2::integer-list-fix l)))))
               0))
  :hints(("Goal"
          :smtlink
          (:fty (acl2::integer-list))))
  :rule-classes nil)
```

### Why All the `fix` Functions?

Z3 lists are typed, so polymorphic functions like `car` need type information. The `integer-list-fix` functions tell SMTLink:
- This is an `integer-list`
- Use the integer-list version of `car`/`cdr`

**Requirement**: `deflist` types must be `true-listp`.

## Example 3: defalist (Association Lists)

```lisp
;; Define a symbol-to-integer mapping
(defalist symbol-integer-alist
  :key-type symbolp
  :val-type integerp
  :true-listp t)

(defthm fty-defalist-theorem
  (implies (and (symbol-integer-alist-p l)
                (symbolp s1)
                (symbolp s2)
                (not (equal (assoc-equal s1 (symbol-integer-alist-fix l))
                            (smt::magic-fix 'symbolp_integerp nil)))
                (not (equal (assoc-equal s2 (symbol-integer-alist-fix l))
                            (smt::magic-fix 'symbolp_integerp nil))))
           (>= (x^2+y^2
                (cdr (smt::magic-fix 'symbolp_integerp
                                     (assoc-equal s1 (symbol-integer-alist-fix l))))
                (cdr (smt::magic-fix 'symbolp_integerp
                                     (assoc-equal s2 (symbol-integer-alist-fix l)))))
               0))
  :hints(("Goal"
          :smtlink
          (:fty (symbol-integer-alist))))
  :rule-classes nil)
```

### The `magic-fix` Function

`smt::magic-fix` provides type annotations for `assoc-equal` results:
- First argument: type name (e.g., `'symbolp_integerp`)
- Second argument: the expression to fix
- Tells SMTLink the pair type returned by `assoc-equal`

## Counter-Examples for defalist

```
Possible counter-example found:
((S2 (SYM 2)) (L (K SYMBOL (SOME 0))) (S1 (SYM 1)))
```

The `(K SYMBOL (SOME 0))` notation means:
- A constant Z3 array
- Maps from symbols to `maybe-integer`
- Every key maps to `(SOME 0)`

`SYM` are generated symbols for counter-examples.

## Example 4: defoption (Optional Types)

```lisp
;; Define maybe-integer (from basictypes.lisp)
(defoption maybe-integer integerp)

;; A function that handles optional integers
(define x^2+y^2-fixed ((x maybe-integer-p)
                       (y maybe-integer-p))
  :returns (res maybe-integer-p)
  (b* ((x (maybe-integer-fix x))
       (y (maybe-integer-fix y))
       ((if (equal x (maybe-integer-fix nil)))
        (maybe-integer-fix nil))
       ((if (equal y (maybe-integer-fix nil)))
        (maybe-integer-fix nil)))
    (maybe-integer-some
     (+ (* (maybe-integer-some->val x)
           (maybe-integer-some->val x))
        (* (maybe-integer-some->val y)
           (maybe-integer-some->val y))))))

(defthm fty-defoption-theorem
  (implies (and (maybe-integer-p m1)
                (maybe-integer-p m2)
                (not (equal m1 (maybe-integer-fix nil)))
                (not (equal m2 (maybe-integer-fix nil))))
           (>= (maybe-integer-some->val (x^2+y^2-fixed m1 m2))
               0))
  :hints(("Goal"
          :smtlink
          (:fty (maybe-integer))))
  :rule-classes nil)
```

### Working with Optional Types:

- `maybe-integer-fix nil` represents the "none" case
- `maybe-integer-some` constructs a "some" value
- `maybe-integer-some->val` extracts the integer

## Summary: FTY with SMTLink

| FTY Type | Hint | Key Functions |
|----------|------|---------------|
| `defprod` | `:fty (typename)` | `typename->field`, `typename-p` |
| `deflist` | `:fty (typename)` | `typename-fix`, `car`, `cdr` |
| `defalist` | `:fty (typename)` | `typename-fix`, `assoc-equal`, `magic-fix` |
| `defoption` | `:fty (typename)` | `typename-fix`, `typename-some`, `typename-some->val` |

### Best Practices:

1. **Always use fix functions** - SMTLink needs type information
2. **Use `magic-fix` for pairs** - `assoc-equal` results need annotation
3. **deflist requires true-listp** - SMTLink checks this
4. **Counter-examples are typed** - Shows concrete FTY values

## Files Referenced

- `smtlink/examples/examples.lisp` - FTY examples
- `smtlink/examples/basictypes.lisp` - `maybe-integer` definition
- `smtlink/examples/util.lisp` - Helper FTY types